In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../train/mimiciii_df.pkl')
df.head()

,SUBJECT_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,...,ICD9_CODE,12h_obs,24h_obs,ID,code_name,code_offset,value,uom,code_order,seq_len
0,1027140,3011529,carevue,CCU,CCU,57,57,2152-07-23 05:34:12,2152-07-24 18:37:32,1.5440,...,NaN,2152-07-23 17:34:12,2152-07-24 05:34:12,3011529,"[Anion Gap, Bicarbonate, Bilirubin, Total, Cho...","[78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 7...","[11.0, 25.0, 0.3, 5.9, 31.0, 108.0, 183.0, 0.7...","[mEq/L, mEq/L, mg/dL, Ratio, mg/dL, mg/dL, mg/...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20
1,1027482,3011175,carevue,MICU,MICU,12,12,2162-03-20 17:22:48,2162-03-22 18:26:10,2.0440,...,NaN,2162-03-21 05:22:48,2162-03-21 17:22:48,3011175,"[30060, Levofloxacin, Dextrose 5%, NS, Fentany...","[337, 397, 397, 397, 397, 397, 397, 397, 397, ...","[ , 500, 100, 100, 200, 3, 250, 500, 50, 1000,...","[ , mg, ml, ml, mcg, mg, mg, mg, ml, ml, ml, m...","[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...",36
2,1011746,3013234,carevue,MICU,MICU,52,52,2107-07-28 17:22:31,2107-08-01 11:03:44,3.7370,...,NaN,2107-07-29 05:22:31,2107-07-29 17:22:31,3013234,"[Acetaminophen, Alanine Aminotransferase (ALT)...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[ , 20.0, 4.0, 64.0, 14.0, 30.0, , , 23.0, 0...","[ug/mL, IU/L, g/dL, IU/L, mEq/L, IU/L, , , m...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",105
3,1010325,3022526,metavision,MICU,MICU,52,52,2145-02-06 02:15:08,2145-02-07 18:47:37,1.6892,...,NaN,2145-02-06 14:15:08,2145-02-07 02:15:08,3022526,"[Alanine Aminotransferase (ALT), Alkaline Phos...","[222, 222, 222, 222, 222, 222, 222, 222, 222, ...","[17.0, 62.0, 11.0, 19.0, 25.0, 1.2, 8.5, 108.0...","[IU/L, IU/L, mEq/L, IU/L, mEq/L, mg/dL, mg/dL,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",19
4,1012708,3035542,metavision,SICU,SICU,33,33,2175-02-20 07:41:28,2175-02-23 10:21:01,3.1108,...,NaN,2175-02-20 19:41:28,2175-02-21 07:41:28,3035542,"[Anion Gap, Bicarbonate, Chloride, Creatine Ki...","[91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 9...","[15.0, 24.0, 98.0, 3.0, 1.8, 3.9, 133.0, 1.0, ...","[mEq/L, mEq/L, mEq/L, ng/mL, mg/dL, mEq/L, mEq...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, ...",30


In [2]:
print(df.columns)

Index(['SUBJECT_ID', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG',
       'age', 'readmission', 'mortality', 'los_3day', 'los_7day', 'ICD9_CODE',
       '12h_obs', '24h_obs', 'ID', 'code_name', 'code_offset', 'value', 'uom',
       'code_order', 'seq_len'],
      dtype='object')


In [6]:
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def embed(input_text):
    tokens = tokenizer(input_text, return_tensors="pt")
    embeddings = model(**tokens).last_hidden_state
    pooled_embeddings = embeddings.mean(dim=1)
    return pooled_embeddings

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading: 100%|██████████| 436M/436M [00:31<00:00, 13.9MB/s] 
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import torch

def embed_data(df):
    # Create empty numpy array with len(df) rows and 256 columns
    df_embed = []
    for k in range(len(df)):
        df_embed.append([])
        for n in range(len(df['code_name'][k])):
            code_name = str(df['code_name'][k][n])
            value = str(df['value'][k][n])
            uom = str(df['uom'][k][n])
            features = [code_name, value, uom]
            # Embed features separately then concatenate the tensors
            embed_input = torch.cat([embed(feature) for feature in features])
            # Save to df
            df_embed[k].append(embed_input)
    return pd.DataFrame(df_embed)

# Test with first 10 rows
df_test = df.head(2)
df_test = embed_data(df_test)
df_test.head()

# Save to pickle
#df_test.to_pickle('df_test.pkl')

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,"[[tensor(0.3046, grad_fn=<UnbindBackward0>), t...","[[tensor(-0.0359, grad_fn=<UnbindBackward0>), ...","[[tensor(0.4241, grad_fn=<UnbindBackward0>), t...","[[tensor(-0.0262, grad_fn=<UnbindBackward0>), ...","[[tensor(0.4338, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2495, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2482, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2935, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2651, grad_fn=<UnbindBackward0>), t...","[[tensor(0.1444, grad_fn=<UnbindBackward0>), t...",...,"[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t..."
1,"[[tensor(0.0869, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3569, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2128, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3085, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2360, grad_fn=<UnbindBackward0>), t...","[[tensor(0.0925, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3569, grad_fn=<UnbindBackward0>), t...","[[tensor(0.1535, grad_fn=<UnbindBackward0>), t...","[[tensor(0.2128, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3085, grad_fn=<UnbindBackward0>), t...",...,"[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t...","[[tensor(0.3680, grad_fn=<UnbindBackward0>), t..."
